# [모듈 2.1] 모델 훈련 스텝 및 모델 등록 스텝 개발 (SageMaker Model Building Pipeline 훈련 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 모델 훈련 및 모델 등록 개요 
- 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 2. 모델 훈련 코드 확인
- 3. 모델 훈련 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 훈련 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
        - (2) 세이지메이커 호스트 모드(로컬 다커 컨테이너 사용) 및 실험(Experiment)사용하여 훈련 코드 실행
        - (3) [필수] SageMaker Model Building Pipeline 에서 모델 훈련 스텝 및 모델 등록 스텝  실행
    

---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---



# 0. 모델 훈련 및 모델 등록 개요
- 모델 훈련에 대한 부분은 scratch/4.1.Train-Pipeline.ipynb 노트북을 참조 바랍니다.
- 모델 등록 스텝은 여기를 참조 바랍니다. --> [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)

# 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
---

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")

%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
# %store  

In [2]:
! aws s3 ls {train_preproc_data_uri} --recursive

2022-06-23 10:57:26     682602 sagemaker-pipeline-step-by-step-phase02/preporc/train.csv


In [3]:
train_prep_df = pd.read_csv(train_preproc_data_uri)
train_prep_df

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


### 기본 훈련 변수 및 하이퍼파라미터 설정

기본 하이퍼파라미터 외에 `scale_pos_weight` 는 레이블이 뷸균형이 되어 있을 경우에, 사용할 수 있는 하나의 방법 입니다. 일반적인 경우에 잘 작동합니다.

In [4]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = project_prefix

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1

def get_pos_scale_weight(df, label):
    '''
    1, 0 의 레이블 분포를 계산하여 클래스 가중치 리턴
    예: 1: 10, 0: 90 이면 90/10 = 9 를 제공함. 
    호출:
        class_weight = get_pos_scale_weight(train_prep_df, label='fraud')
    '''
    fraud_sum = df[df[label] == 1].shape[0]
    non_fraud_sum = df[df[label] == 0].shape[0]
    class_weight = int(non_fraud_sum / fraud_sum)
    print(f"fraud_sum: {fraud_sum} , non_fraud_sum: {non_fraud_sum}, class_weight: {class_weight}")
    return class_weight
    
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')


hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "alpha" : "0.2", 
        "eta": "0.333",
        "min_child_weight": "7",
        "objective": "binary:logistic",
        "num_round": "100",
}



fraud_sum: 131 , non_fraud_sum: 3869, class_weight: 29


# 2. 모델 훈련 코드 확인

훈련 코드는 크게 아래와 같이 구성 되어 있습니다.
- 커맨드 인자로 전달된 변수 내용 확인
- 훈련 데이터를 로딩 합니다.
- xgboost의 cross-validation(cv) 로 훈련 합니다.
- 훈련 및 검증 데이터 세트의 roc-auc 값을 metrics_data 에 저장
    - 모델 훈련시 생성되는 지표(예: loss 등)는 크게 두가지 방식으로 사용할 수 있습니다.
        - 클라우드 워치에서 실시간으로 지표 확인
        - 하이퍼 파라미터 튜닝(HPO) 에서 평가 지표로 사용 (예: validation:roc-auc)
        - 참조 --> [Monitor and Analyze Training Jobs Using Metrics](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html)
        - 참조: XGBoost Framework 에는 이미 디폴트로 정의된 metric definition이 있어서, 정의된 규칙에 따라서 모델 훈련시에 print() 를 하게 되면, metric 이 클라우드 워치 혹은 HPO에서 사용이 가능합니다.
        
            
```            
Name				Regex
validation:auc	.*\[[0-9]+\].*#011validation-auc:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*
train:auc	    .*\[[0-9]+\].*#011train-auc:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*

실제 코드에 위의 Regex 형태로 print() 반영
print(f"[0]#011train-auc:{train_auc_mean}")
print(f"[1]#011validation-auc:{validation_auc_mean}")
```
    
- 훈련 성능을 나타내는 지표를 저장합니다. (metrics.json)
- 훈련이 모델 아티펙트를 저장 합니다.

---

In [5]:
!pygmentize src/xgboost_script.py

import os
import sys
import pickle
import xgboost as xgb
import argparse
import pandas as pd
import json

import pandas as pd
pd.options.display.max_rows=20
pd.options.display.max_columns=10

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    ###################################
    ## 커맨드 인자 처리
    ###################################    
    
    # Hyperparameters are described here
    parser.add_argument('--scale_pos_weight', type=int, default=50)    
    parser.add_argument('--num_round', type=int, default=100)
    parser.add_argument('--max_depth', type=int, default=3)
    parser.add_argument('--min_child_weight', type=float, default=1.0)    
    parser.add_argument('--alpha', type=float, default=0.0)        
    parser.add_argument('--eta', type=float, default=0.2)
    parser.add_argument('--objective', type=str, default='binary:logistic')
    parser.add_argument('--nfold', type=int, default=5)
    parser.add_argument('--early_stopping_rounds', type=int, defaul

# 3. 모델 훈련 스텝 개발 및 실행
---



## (1) 로컬 노트북 인스턴스에서 로컬 모드(로컬 다커 컨테이너 사용)로 훈련 코드 실행


#### [참조] 로컬모드로 실행시에 403 Forbidden Error 및 Credential 에러  발생시
- [로컬 모드 실행시 403 에러 해결 방법](../scratch/error/Trouble-Shooting.ipynb)

In [6]:

xgb_estimator_local = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    session = sagemaker.LocalSession(),    
    instance_count = train_instance_count,
    instance_type = 'local',
    framework_version = "1.0-1"
)
    
xgb_estimator_local.fit(inputs = {'train': train_preproc_data_uri},
                      wait=True,                        
                 )


Creating wiugvytgj9-algo-1-0kdph ... 
Creating wiugvytgj9-algo-1-0kdph ... done
Attaching to wiugvytgj9-algo-1-0kdph
wiugvytgj9-algo-1-0kdph | INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
wiugvytgj9-algo-1-0kdph | INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
wiugvytgj9-algo-1-0kdph | INFO:sagemaker_xgboost_container.training:Invoking user training script.
wiugvytgj9-algo-1-0kdph | INFO:sagemaker-containers:Module xgboost_script does not provide a setup.py. 
wiugvytgj9-algo-1-0kdph | Generating setup.py
wiugvytgj9-algo-1-0kdph | INFO:sagemaker-containers:Generating setup.cfg
wiugvytgj9-algo-1-0kdph | INFO:sagemaker-containers:Generating MANIFEST.in
wiugvytgj9-algo-1-0kdph | INFO:sagemaker-containers:Installing module with the following command:
wiugvytgj9-algo-1-0kdph | /miniconda3/bin/python3 -m pip install . 
wiugvytgj9-algo-1-0kdph | Processing /opt/ml/code
wiugvytgj9-algo-1-0kdph | Building wheels for collected package

Failed to delete: /tmp/tmpn8n1g2lg/algo-1-0kdph Please remove it manually.


===== Job Complete =====


## (2) 세이지메이커 호스트 모드(로컬 다커 컨테이너 사용) 및 실험(Experiment)사용하여 훈련 코드 실행

### 실험(Experiment) 세팅
- Amazon SageMaker 실험은 기계 학습 실험을 구성, 추적, 비교 및 평가할 수 있는 Amazon SageMaker 의 기능입니다
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 실험을 통한 Machine Learning 관리](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/experiments.html)
- sagemaker experiment는 추가적인 패키지를 설치하여야 합니다. 0.0.Setup-Environment 가 실행이 안되었다고 하면, `!pip install --upgrade sagemaker-experiments` 를 통해 설치 해주세요.
- 여기서는 boto3 API를 통해서 실험을 생성합니다. SageMaker Python SDK를 통해서도 가능합니다.


In [7]:
# !pip install --upgrade sagemaker-experiments

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime

sm = boto3.client('sagemaker')


# 설험에 대한 이름을 생성 합니다.
experiment_name = project_prefix + '-single-train'

# 실험이 존재하지 않으면 생성하고, 그렇지 않으면 지나갑니다.
try:
    response = sm_client.describe_experiment(ExperimentName=experiment_name)
    print(f"Experiment:{experiment_name} already exists")    
    
except:
    response = sm_client.create_experiment(
        ExperimentName = experiment_name,
        Description = 'Experiment for fraud detection',
    )
    print(f"Experiment:{experiment_name} is created")        


Experiment:sagemaker-pipeline-step-by-step-phase02-single-train already exists


### 하이퍼 파리미터 변경 실험
- max_depth 5개의 값을 바꾸면서 5개의 훈련잡을 실행합니다.
    - ```for i, max_depth_num in enumerate([1,3,5,7,10]):```
    - 만약에 리소스 제한의 에러가 발생하면, 5개를 2개 정도로 줄여서 실행 해주세요.
- 위에서 생성한 Experiment 안에 5개의 Trial(시도) 를 생성합니다.
- xgb_estimator 에 각각의 하이파라미터를 인자로 제공합니다.
- xgb_estimator.fit()에 Experiment의 설정 파일을 제공합니다.
    - 1개의 실험, 각각의 시도가 설정되어 훈련을 시작 합니다.

In [8]:
instance_type = 'ml.m5.xlarge'
for i, max_depth_num in enumerate([1,3,5,7,10]):
    hyperparameters = {
           "scale_pos_weight" : class_weight,    
            "max_depth": f"{max_depth_num}",
            "alpha" : "0", 
            "eta": "0.3",
            "min_child_weight": "1",
            "objective": "binary:logistic",
            "num_round": "100",
    }
    
    # 시도 이름 생성
    ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
    trial_name = experiment_name + f"-{ts}"

    # 1개의 실험 안에 시도를 생성함.
    response = sm_client.create_trial(
        ExperimentName = experiment_name,
        TrialName = trial_name,
    )    
    
    # 실험 설정: 실험 이름, 시도 이름으로 구성
    experiment_config = {
        'ExperimentName' : experiment_name,
        'TrialName' : trial_name,
        "TrialComponentDisplayName" : 'Training',
    }    

    
    xgb_estimator = XGBoost(
        entry_point = "xgboost_script.py",
        source_dir = "src",
        output_path = estimator_output_path,
        code_location = estimator_output_path,
        hyperparameters = hyperparameters,
        role = role,
        instance_count = train_instance_count,
        instance_type = instance_type,
        framework_version = "1.0-1")

    xgb_estimator.fit(inputs = {'train': train_preproc_data_uri},
                          experiment_config = experiment_config, # 실험 설정 제공
                          wait=False,                        
                     )

    



INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2022-06-23-12-25-27-002
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2022-06-23-12-25-27-438
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2022-06-23-12-25-27-868
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2022-06-

In [9]:
## 마지막 estimator의 로그 출력
xgb_estimator.logs()

2022-06-23 12:25:31 Starting - Starting the training job...ProfilerReport-1655987131: InProgress
......
2022-06-23 12:26:55 Starting - Preparing the instances for training......
2022-06-23 12:27:55 Downloading - Downloading input data...
2022-06-23 12:28:29 Training - Downloading the training image.....INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Created wheel for xgboost-script: filename=xgboost_script-1.0.0-py2.py3-none-any.whl size=31659 sha256=0aea7aa7b7d7310f4929357ce87e18b

###  실험 결과 보기
위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [10]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 5
pd.options.display.max_colwidth = 50

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
)

trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,alpha,eta,max_depth,min_child_weight,num_round,objective,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,scale_pos_weight,train:auc - Min,train:auc - Max,train:auc - Avg,train:auc - StdDev,train:auc - Last,train:auc - Count,validation:auc - Min,validation:auc - Max,validation:auc - Avg,validation:auc - StdDev,validation:auc - Last,validation:auc - Count,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,sagemaker-xgboost-2022-06-23-12-25-30-616-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""7""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2022-06-23-12-25-30-616""","""xgboost_script.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/sagemak...",29.0,0.9155,0.9155,0.9155,0.0,0.9155,1,0.7855,0.7855,0.7855,0.0,0.7855,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...
1,sagemaker-xgboost-2022-06-23-12-25-31-125-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""10""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2022-06-23-12-25-31-125""","""xgboost_script.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/sagemak...",29.0,0.9443,0.9443,0.9443,0.0,0.9443,1,0.7802,0.7802,0.7802,0.0,0.7802,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,sagemaker-xgboost-2022-06-23-11-02-31-320-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""7""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2022-06-23-11-02-31-320""","""xgboost_script.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/sagemak...",29.0,0.9155,0.9155,0.9155,0.0,0.9155,1,0.7855,0.7855,0.7855,0.0,0.7855,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...
14,sagemaker-xgboost-2022-06-23-11-02-30-211-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""5""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2022-06-23-11-02-30-211""","""xgboost_script.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/sagemak...",29.0,0.8858,0.8858,0.8858,0.0,0.8858,1,0.7985,0.7985,0.7985,0.0,0.7985,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...


### 모델 평가 지표에 순서에 따른 시도 보기
- 아래는 모델 평가 지표에 따른 순서로 보여주기 입니다.

In [11]:

trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
    sort_by="metrics.validation:auc.max",        
    sort_order="Descending",
    metric_names=["validation:auc"],
    parameter_names=["hidden_channels", "epochs", "dropout", "optimizer"],
)

trial_component_training_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,validation:auc - Min,validation:auc - Max,validation:auc - Avg,validation:auc - StdDev,validation:auc - Last,validation:auc - Count,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,sagemaker-xgboost-2022-06-23-11-02-28-398-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,0.8257,0.8257,0.8257,0.0,0.8257,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...
1,sagemaker-xgboost-2022-06-23-11-23-59-125-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,0.8257,0.8257,0.8257,0.0,0.8257,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,sagemaker-xgboost-2022-06-23-11-02-31-720-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,0.7802,0.7802,0.7802,0.0,0.7802,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...
14,sagemaker-xgboost-2022-06-23-11-24-05-991-aws-...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,0.7802,0.7802,0.7802,0.0,0.7802,1,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,None,s3://sagemaker-us-east-1-057716757052/sagemake...,[sagemaker-pipeline-step-by-step-phase02-singl...,[sagemaker-pipeline-step-by-step-phase02-singl...


In [12]:
train_exp_experiment_name = experiment_name
%store train_exp_experiment_name

Stored 'train_exp_experiment_name' (str)


### (2) [추가] 모델 평가 파일 다운로드

- 아래 작업은 모델 훈련을 하면서 모델의 평가 (Train:Auc, Validation:Auc) 의 값을 'metrics.json' 에 저장을 하고, 도커 안의 환경 변수가 가르키는 곳에  (SM_OUTPUT_DATA_DIR=/opt/ml/output/data) 저장을 합니다.
- metrics.json 파일을 훈련이 끝난 후에 아래와 같은 경로 (에: s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/sagemaker-xgboost-2022-06-23-12-25-31-125/output/output.tar.gz) 에 업로딩 합니다. 
- 이 파일을 로컬 폴더 (에: ./output) 에 다운로드 하고, 압축을 해제 합니다.
- 'metrics.json' 파일을 S3 에 업로드 하여 s3_uri 경로를 저장 합니다.
- s3_uri 는 model registry 에 모델을 등록시에 인자로 제공하여 , 모델의 성능 메트릭을 관리 합니다.

In [13]:
output_artifact_path = f"{xgb_estimator.output_path}/{xgb_estimator.latest_training_job.job_name}/output/output.tar.gz"
print(f"output_artifact_path: {output_artifact_path}")

output_artifact_path: s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/sagemaker-xgboost-2022-06-23-12-25-31-125/output/output.tar.gz


In [14]:
! aws s3 ls {output_artifact_path}

2022-06-23 12:29:29        182 output.tar.gz


In [15]:
output_data_dir = './output'
os.makedirs(output_data_dir, exist_ok=True)

In [16]:
%%sh -s {output_artifact_path} {output_data_dir}

output_artifact_path=$1
output_data_dir=$2

echo $output_artifact_path
echo $output_data_dir


# 모델을 S3에서 로컬로 다운로드
aws s3 cp $output_artifact_path $output_data_dir

# # 모델 다운로드 폴더로 이동
cd $output_data_dir

# # 압축 해제
tar -xvf output.tar.gz  

rm -rf output.tar.gz  

s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/sagemaker-xgboost-2022-06-23-12-25-31-125/output/output.tar.gz
./output
download: s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/sagemaker-xgboost-2022-06-23-12-25-31-125/output/output.tar.gz to output/output.tar.gz
metrics.json


로컬의 metrics.json 을 S3에 업로딩 합니다.

In [17]:
metric_path = os.path.join(output_data_dir, 'metrics.json')
metric_data_uri = f"s3://{default_bucket}/{project_prefix}"

metric_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=metric_path, 
    desired_s3_uri=metric_data_uri,    
)
print(metric_data_uri)



s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/metrics.json


## (3) SageMaker Pipeline에서  실행 
- 모델 훈련 스텝과 모델 등록 스텝 두가지를 실행합니다.
    - 두 개의 단계가 서로 의존성이 있기에, 두 개의 단계를 연결을 합니다.

---



### 모델 빌딩 파이프라인 변수 생성



In [18]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)


training_instance_count = ParameterInteger(
    name="TrainInstanceCount",
    default_value=1
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [19]:
hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "alpha" : "0", 
        "eta": "0.3",
        "min_child_weight": "1",
        "objective": "binary:logistic",
        "num_round": "100",
}

training_instance_type = "ml.m5.xlarge"

xgb_train = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = training_instance_type,    
    framework_version = "1.0-1")

이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.

In [20]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_data_uri,
            content_type="text/csv"
        ),
    },
)

### 모델 등록 스텝
- 모델의 성능 지표 (metrics.json) 를 등록하기 위해 ModelMetrics 오브젝트 생성.
    - [Python SDK, class sagemaker.model_metrics.ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.MetricsSource)
- 모델들이 저장이 될 그룹 이름을 제공하고, 모델 등록 스텝을 정의 합니다.
- 모델 등록 단계의 개발자 가이드 
    - [모델 등록기 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-register-model)
    - [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)
- 모델 그룹 리스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)   

In [21]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 

# 위의 step_eval 에서 S3 로 올린 evaluation.json 파일안의 지표를 "모델 레지스트리" 에 모데 버전 등록시에 삽입함
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=metric_data_uri,
        content_type="application/json"
    )
)



In [22]:
model_package_group_name = f"{project_prefix}"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

from sagemaker.workflow.step_collections import RegisterModel

step_register = RegisterModel(
    name= f"FraudRegisterModel",
    estimator=xgb_train,
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,    
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics, # 모델 썽능 등록
)

### 모델 빌딩 파이프라인 정의
- 파이프라인과 실험(Experiment)가 통합이 되었습니다. 이를 위한 실험 설정 파일을 같이 제공합니다.

In [23]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig


pipeline_name = project_prefix + "-Train-Exp"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        input_data,
        model_approval_status,
    ],
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_train, step_register],
)

In [24]:
import json

definition = json.loads(pipeline.definition())
# definition

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [25]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase02-train-exp',
 'ResponseMetadata': {'RequestId': '7ae51397-c520-47f9-9d81-08fed4a6774a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7ae51397-c520-47f9-9d81-08fed4a6774a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '117',
   'date': 'Thu, 23 Jun 2022 12:30:17 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [26]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [27]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase02-train-exp',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase02-train-exp/execution/ml3oxlzc60pd',
 'PipelineExecutionDisplayName': 'execution-1655987418110',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 6, 23, 12, 30, 17, 983000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 6, 23, 12, 30, 17, 983000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'c268ba72-c011-43aa-b614-21cea16db9b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c268ba72-c011-43aa-b614-21cea16db9b8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '463',
   'date': 'Thu, 23 Jun 2022 12:30:17 GMT'},
  'RetryAttempts': 0}}

In [28]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [29]:
execution.list_steps()

[{'StepName': 'FraudRegisterModel',
  'StartTime': datetime.datetime(2022, 6, 23, 12, 33, 31, 798000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 12, 33, 32, 525000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/4'}}},
 {'StepName': 'FraudTrain',
  'StartTime': datetime.datetime(2022, 6, 23, 12, 30, 19, 497000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 12, 33, 31, 86000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:training-job/pipelines-ml3oxlzc60pd-fraudtrain-ipsao6lgto'}}}]

## 모델 레지스트리에서 모델 등록 확인
위에서 등록한 모델 그룹 이름을 통해서 어떤 모델이 등록되었는지를 확인 합니다.
스튜디오에서 실제 등록된 성능 지표를 확인 할 수 있습니다.


![studio_model_metric_registry.png](img/studio_model_metric_registry.png)

- 등록된 모델 버전에 대한 보기 --> [모델 버전의 세부 정보 보기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry-details.html)

In [30]:
# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)
response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase02',
   'ModelPackageVersion': 4,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/4',
   'CreationTime': datetime.datetime(2022, 6, 23, 12, 33, 32, 395000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase02',
   'ModelPackageVersion': 3,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/3',
   'CreationTime': datetime.datetime(2022, 6, 23, 12, 4, 8, 945000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase02',
   'ModelPackageVersion': 2,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-pac

#### 등록된 모델 버전의 상세 정보 확인

In [31]:
ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

{'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase02',
 'ModelPackageVersion': 4,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/4',
 'CreationTime': datetime.datetime(2022, 6, 23, 12, 33, 32, 395000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
    'ImageDigest': 'sha256:04889b02181f14632e19ef6c2a7d74bfe699ff4c7f44669a78834bc90b77fe5a',
    'ModelDataUrl': 's3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/pipelines-ml3oxlzc60pd-FraudTrain-IpSaO6lgTO/output/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m5.xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.t2.medium', 'ml.m5.xlarge'],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': 

### 아티펙트 경로 추출
위의 훈련 스텝이 완료되면 실행해주세요

In [32]:
from IPython.display import display as dp
def get_train_artifact(execution, client, job_type,  kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()
    dp("response: ", response)
    proc_arn = response[1]['Metadata'][job_type]['Arn'] # 1은 훈련 스텝
    train_job_name = proc_arn.split('/')[-1]
    # print("train_job_name: ", train_job_name)
    response = client.describe_training_job(TrainingJobName = train_job_name)
    # print("\nresponse: ", response)    
    train_model_artifact = response['ModelArtifacts']['S3ModelArtifacts']    
    
    return train_model_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: ", train_model_artifact)


'response: '

[{'StepName': 'FraudRegisterModel',
  'StartTime': datetime.datetime(2022, 6, 23, 12, 33, 31, 798000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 12, 33, 32, 525000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/4'}}},
 {'StepName': 'FraudTrain',
  'StartTime': datetime.datetime(2022, 6, 23, 12, 30, 19, 497000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 12, 33, 31, 86000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:training-job/pipelines-ml3oxlzc60pd-fraudtrain-ipsao6lgto'}}}]

 train_model_artifact:  s3://sagemaker-us-east-1-057716757052/sagemaker-pipeline-step-by-step-phase02/training_jobs/pipelines-ml3oxlzc60pd-FraudTrain-IpSaO6lgTO/output/model.tar.gz


In [33]:
image_uri = xgb_train.image_uri

훈련 모델 아티펙트와, 훈련시 사용한 다커 이미지의 경로를 저장 합니다.

In [34]:
%store train_model_artifact
%store image_uri
%store model_package_group_name

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
Stored 'model_package_group_name' (str)


# 4. 리소스 정리: 파이프라인
- 위에서 생성한 파이프라인을 제거 합니다.
- isDeletePipeline=False, verbose=Fasle
    - 파이프라인을 지우지 않고, 존재하는지 확인 합니다.
- isDeletePipeline=False, verbose=True
    - 파이프라인의 정의를 자세하 확인 합니다.
- isDeletePipeline=True, verbose=True or False
    - 파이프라인을 삭제 합니다.

In [35]:
from src.p_utils import clean_pipeline

# clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=True, verbose=False)   

pipeline sagemaker-pipeline-step-by-step-phase02-Train-Exp exists
pipeline sagemaker-pipeline-step-by-step-phase02-Train-Exp is deleted
